### Importing of modules and packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


In [ ]:
df_postive_cases = pd.read_excel(r'/Users/ivan/Desktop/PycharmProjects/Data_Science_Mini/Data/raw/covid19infectionsurveydatasets2020122423122020174305.xlsx',
                                 sheet_name='1a',
                                 engine='openpyxl')

# remove unwanted rows
df_postive_cases = df_postive_cases[17:41].reindex()

# remove unwanted columns
column_list = ['Unnamed: 1', 'Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 6','Unnamed: 8','Unnamed: 9',]
df_postive_cases = df_postive_cases.drop(column_list, axis=1)

# remove empty columns
df_postive_cases = df_postive_cases.loc[:,:'Unnamed: 7']

# rename columns
df_postive_cases = df_postive_cases.rename(columns={'Contents': 'Date',
                                                    'Unnamed: 4': 'Estimate of number testing positive',
                                                    'Unnamed: 7': 'Proportion of population that had COVID'})

In [ ]:
# function to turn ratio (like '1 in 2000') into a decimal
def convert_ratio(x):
    a,b = x.split('in')
    c = int(a) / int(b)
    return c

In [ ]:
# lambda function to remove the commas (for example replace 1,200 with 1200)
df_postive_cases['Proportion of population that had COVID'] = df_postive_cases['Proportion of population that had COVID'].map(lambda x: x.replace(',',''))

df_postive_cases['Proportion of population that had COVID'] = df_postive_cases['Proportion of population that had COVID'].apply(convert_ratio)
df_postive_cases.reset_index(drop=True, inplace=True)
df_postive_cases

In [ ]:
# function to convert '28 November' to 28/11/2020
# default year of 1900 needs replacing
def convert_date(x):
    x = datetime.strptime(x,"%d %B").strftime('%d/%m/%Y')
    x = x.replace('1900','2020')
    return x

In [ ]:
# make a copy of the Series (column named 'Date') to avoid the warning 'A value is trying to be set on a copy of a slice from a DataFrame'
df_copy = df_postive_cases['Date'].copy()

# iterate over 'Date' series and replace with last date (i.e. 06 July to 12 July replaced with 12/07/2020)

for i in range(df_postive_cases['Date'].size):
    date = df_postive_cases['Date'].iloc[i].split(' to ')[1]
    date = date.strip()
    df_copy.iloc[i] = convert_date(date)

# reassign Series back into dataframe
df_postive_cases['Date'] = df_copy

# colour the values in the style of a bar chart
df_postive_cases.style.bar(subset=['Estimate of number testing positive'], align='mid', color=['#ffAAAA'])